In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
import sklearn
import sklearn.cluster as cluster
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [35]:
df = pd.read_excel('./Datasets/dataset_complete_cluster.xlsx')

In [36]:
df

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,peakrpm,citympg,highwaympg,price,sales 2018,sales 2019,sales 2020,sales 2021,total sales,cluster
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,5000,21,27,13495.0,34300.0,18294.0,17200.0,18252.0,88046,No Segment
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,5000,21,27,16500.0,12043.0,9444.0,10282.0,10539.0,42308,No Segment
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,5000,19,26,16500.0,17075.0,10932.0,7436.0,6297.0,41740,Regular I
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,5500,24,30,13950.0,0.0,0.0,0.0,0.0,0,Regular II
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,5500,18,22,17450.0,0.0,0.0,0.0,0.0,0,Luxury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,5400,23,28,16845.0,0.0,0.0,0.0,0.0,0,Regular II
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,5300,19,25,19045.0,0.0,0.0,0.0,0.0,0,Luxury
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,5500,18,23,21485.0,0.0,0.0,0.0,0.0,0,Luxury
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,4800,26,27,22470.0,0.0,0.0,0.0,0.0,0,Regular II


### removing unnecessary columns that don't bear a linear relationship with sales like car_ID, carname etc.

In [51]:
df.drop(['car_ID'], axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fueltype        205 non-null    object 
 1   aspiration      205 non-null    object 
 2   doornumber      205 non-null    object 
 3   carbody         205 non-null    object 
 4   drivewheel      205 non-null    object 
 5   curbweight      205 non-null    int64  
 6   enginetype      205 non-null    object 
 7   cylindernumber  205 non-null    object 
 8   enginesize      205 non-null    int64  
 9   fuelsystem      205 non-null    object 
 10  horsepower      205 non-null    int64  
 11  peakrpm         205 non-null    int64  
 12  citympg         205 non-null    int64  
 13  highwaympg      205 non-null    int64  
 14  price           205 non-null    float64
 15  total sales     205 non-null    int64  
dtypes: float64(1), int64(7), object(8)
memory usage: 25.8+ KB


In [50]:

df

,car_ID,fueltype,aspiration,doornumber,carbody,drivewheel,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,horsepower,peakrpm,citympg,highwaympg,price,total sales
0,1,gas,std,2,convertible,rwd,2548,dohc,4,130,mpfi,111,5000,21,27,13495.0,88046
1,2,gas,std,2,convertible,rwd,2548,dohc,4,130,mpfi,111,5000,21,27,16500.0,42308
2,3,gas,std,2,hatchback,rwd,2823,ohcv,6,152,mpfi,154,5000,19,26,16500.0,41740
3,4,gas,std,4,sedan,fwd,2337,ohc,4,109,mpfi,102,5500,24,30,13950.0,0
4,5,gas,std,4,sedan,4wd,2824,ohc,5,136,mpfi,115,5500,18,22,17450.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,gas,std,4,sedan,rwd,2952,ohc,4,141,mpfi,114,5400,23,28,16845.0,0
201,202,gas,turbo,4,sedan,rwd,3049,ohc,4,141,mpfi,160,5300,19,25,19045.0,0
202,203,gas,std,4,sedan,rwd,3012,ohcv,6,173,mpfi,134,5500,18,23,21485.0,0
203,204,diesel,turbo,4,sedan,rwd,3217,ohc,6,145,idi,106,4800,26,27,22470.0,0


### one hot encoding for strings, like enginetype, fueltype, aspiration etc.

In [52]:
columns = [0, 1, 3, 4, 6, 9]
for i in columns:
    onehot_encoder = preprocessing.OneHotEncoder(sparse = False)
    li = onehot_encoder.fit_transform(df.iloc[:, i].values.reshape(205, 1))
    # print(li, len(li))
    for j in range(len(li[0])):
        df.at[:, df.columns[i] + str(j)] = li[:, j]
df

,fueltype,aspiration,doornumber,carbody,drivewheel,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,...,enginetype5,enginetype6,fuelsystem0,fuelsystem1,fuelsystem2,fuelsystem3,fuelsystem4,fuelsystem5,fuelsystem6,fuelsystem7
0,gas,std,2,convertible,rwd,2548,dohc,4,130,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,gas,std,2,convertible,rwd,2548,dohc,4,130,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,gas,std,2,hatchback,rwd,2823,ohcv,6,152,mpfi,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,gas,std,4,sedan,fwd,2337,ohc,4,109,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,gas,std,4,sedan,4wd,2824,ohc,5,136,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,gas,std,4,sedan,rwd,2952,ohc,4,141,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
201,gas,turbo,4,sedan,rwd,3049,ohc,4,141,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
202,gas,std,4,sedan,rwd,3012,ohcv,6,173,mpfi,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
203,diesel,turbo,4,sedan,rwd,3217,ohc,6,145,idi,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fueltype        205 non-null    object 
 1   aspiration      205 non-null    object 
 2   doornumber      205 non-null    object 
 3   carbody         205 non-null    object 
 4   drivewheel      205 non-null    object 
 5   curbweight      205 non-null    int64  
 6   enginetype      205 non-null    object 
 7   cylindernumber  205 non-null    object 
 8   enginesize      205 non-null    int64  
 9   fuelsystem      205 non-null    object 
 10  horsepower      205 non-null    int64  
 11  peakrpm         205 non-null    int64  
 12  citympg         205 non-null    int64  
 13  highwaympg      205 non-null    int64  
 14  price           205 non-null    float64
 15  total sales     205 non-null    int64  
 16  fueltype0       205 non-null    float64
 17  fueltype1       205 non-null    flo

### changing strings to integers

In [46]:
dict = {'two': 2, 'four': 4, 'three' : 3, 'six' : 6, 'eight' : 8, 'twelve' : 12, 'five' : 5}

In [54]:
# for i in range(205):
#     df.at[i, 'cylindernumber'] = dict[df.at[i, 'cylindernumber']]
#     df.at[i, 'doornumber'] = dict[df.at[i, 'doornumber']]
df


,fueltype,aspiration,doornumber,carbody,drivewheel,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,...,enginetype5,enginetype6,fuelsystem0,fuelsystem1,fuelsystem2,fuelsystem3,fuelsystem4,fuelsystem5,fuelsystem6,fuelsystem7
0,gas,std,2,convertible,rwd,2548,dohc,4,130,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,gas,std,2,convertible,rwd,2548,dohc,4,130,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,gas,std,2,hatchback,rwd,2823,ohcv,6,152,mpfi,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,gas,std,4,sedan,fwd,2337,ohc,4,109,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,gas,std,4,sedan,4wd,2824,ohc,5,136,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,gas,std,4,sedan,rwd,2952,ohc,4,141,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
201,gas,turbo,4,sedan,rwd,3049,ohc,4,141,mpfi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
202,gas,std,4,sedan,rwd,3012,ohcv,6,173,mpfi,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
203,diesel,turbo,4,sedan,rwd,3217,ohc,6,145,idi,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [55]:
df.iloc[0, :]

fueltype                  gas
aspiration                std
doornumber                  2
carbody           convertible
drivewheel                rwd
curbweight               2548
enginetype               dohc
cylindernumber              4
enginesize                130
fuelsystem               mpfi
horsepower                111
peakrpm                  5000
citympg                    21
highwaympg                 27
price                 13495.0
total sales             88046
fueltype0                 0.0
fueltype1                 1.0
aspiration0               1.0
aspiration1               0.0
carbody0                  1.0
carbody1                  0.0
carbody2                  0.0
carbody3                  0.0
carbody4                  0.0
drivewheel0               0.0
drivewheel1               0.0
drivewheel2               1.0
enginetype0               1.0
enginetype1               0.0
enginetype2               0.0
enginetype3               0.0
enginetype4               0.0
enginetype

### removing columns which have been transformed by one-hot encoding and are of no use any longer

In [56]:
df.drop(['fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'fuelsystem'], axis = 1, inplace = True)
df.columns

Index(['doornumber', 'curbweight', 'cylindernumber', 'enginesize',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price',
       'total sales', 'fueltype0', 'fueltype1', 'aspiration0', 'aspiration1',
       'carbody0', 'carbody1', 'carbody2', 'carbody3', 'carbody4',
       'drivewheel0', 'drivewheel1', 'drivewheel2', 'enginetype0',
       'enginetype1', 'enginetype2', 'enginetype3', 'enginetype4',
       'enginetype5', 'enginetype6', 'fuelsystem0', 'fuelsystem1',
       'fuelsystem2', 'fuelsystem3', 'fuelsystem4', 'fuelsystem5',
       'fuelsystem6', 'fuelsystem7'],
      dtype='object')

In [57]:
df

,doornumber,curbweight,cylindernumber,enginesize,horsepower,peakrpm,citympg,highwaympg,price,total sales,...,enginetype5,enginetype6,fuelsystem0,fuelsystem1,fuelsystem2,fuelsystem3,fuelsystem4,fuelsystem5,fuelsystem6,fuelsystem7
0,2,2548,4,130,111,5000,21,27,13495.0,88046,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,2548,4,130,111,5000,21,27,16500.0,42308,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,2823,6,152,154,5000,19,26,16500.0,41740,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,2337,4,109,102,5500,24,30,13950.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,2824,5,136,115,5500,18,22,17450.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,4,2952,4,141,114,5400,23,28,16845.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
201,4,3049,4,141,160,5300,19,25,19045.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
202,4,3012,6,173,134,5500,18,23,21485.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
203,4,3217,6,145,106,4800,26,27,22470.0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [97]:
df.columns

Index(['car_ID', 'doornumber', 'curbweight', 'cylindernumber', 'enginesize',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price',
       'total sales', 'fueltype0', 'fueltype1', 'aspiration0', 'aspiration1',
       'carbody0', 'carbody1', 'carbody2', 'carbody3', 'carbody4',
       'drivewheel0', 'drivewheel1', 'drivewheel2', 'enginetype0',
       'enginetype1', 'enginetype2', 'enginetype3', 'enginetype4',
       'enginetype5', 'enginetype6', 'fuelsystem0', 'fuelsystem1',
       'fuelsystem2', 'fuelsystem3', 'fuelsystem4', 'fuelsystem5',
       'fuelsystem6', 'fuelsystem7'],
      dtype='object')

In [58]:
import numpy as np
from sklearn import linear_model

In [60]:
reg = linear_model.LinearRegression()

### renaming 'total sales' to 'totalSales' as the space was causing some issues in the code

In [61]:
df.rename(columns = {'total sales':'totalSales'}, inplace = True)
df

,doornumber,curbweight,cylindernumber,enginesize,horsepower,peakrpm,citympg,highwaympg,price,totalSales,...,enginetype5,enginetype6,fuelsystem0,fuelsystem1,fuelsystem2,fuelsystem3,fuelsystem4,fuelsystem5,fuelsystem6,fuelsystem7
0,2,2548,4,130,111,5000,21,27,13495.0,88046,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,2548,4,130,111,5000,21,27,16500.0,42308,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,2823,6,152,154,5000,19,26,16500.0,41740,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,2337,4,109,102,5500,24,30,13950.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,2824,5,136,115,5500,18,22,17450.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,4,2952,4,141,114,5400,23,28,16845.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
201,4,3049,4,141,160,5300,19,25,19045.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
202,4,3012,6,173,134,5500,18,23,21485.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
203,4,3217,6,145,106,4800,26,27,22470.0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [65]:
df.totalSales

0      88046
1      42308
2      41740
3          0
4          0
       ...  
200        0
201        0
202        0
203        0
204        0
Name: totalSales, Length: 205, dtype: int64

## runnning linear regression with totalSales as the dependent variable
### doornumber, curbweight, cyliindernumber......, fuelsystem7 are independent variables and are affecting the sales linearly

In [62]:
reg.fit(df[['doornumber', 'curbweight', 'cylindernumber', 'enginesize',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price', 'fueltype0', 'fueltype1', 'aspiration0', 'aspiration1',
       'carbody0', 'carbody1', 'carbody2', 'carbody3', 'carbody4',
       'drivewheel0', 'drivewheel1', 'drivewheel2', 'enginetype0',
       'enginetype1', 'enginetype2', 'enginetype3', 'enginetype4',
       'enginetype5', 'enginetype6', 'fuelsystem0', 'fuelsystem1',
       'fuelsystem2', 'fuelsystem3', 'fuelsystem4', 'fuelsystem5',
       'fuelsystem6', 'fuelsystem7']], df.totalSales)
# list of independent variables

LinearRegression()

## linear regression coefficients

In [63]:
reg.coef_

array([ 9.70693607e+03, -3.27567287e+02, -2.52550550e+04,  3.52560947e+02,
        1.74171271e+03, -1.52149162e+02,  3.06049716e+03, -7.93079145e+01,
        9.73400803e+00, -2.78516373e+04,  2.78516373e+04,  2.86272329e+04,
       -2.86272329e+04, -9.02270526e+04,  4.48995343e+03,  2.26344547e+04,
        1.01807534e+04,  5.29218911e+04,  9.61468328e+04, -8.77124822e+04,
       -8.43435058e+03,  7.09395782e+04, -6.69397743e+04,  5.69417564e+04,
        4.49788123e+04, -1.48778282e+05,  1.09678831e+05, -6.68209216e+04,
        1.23312233e+05, -3.45364427e+04,  2.76453054e+04, -2.78516373e+04,
       -2.95947674e+04, -2.73449382e+04,  4.71369255e+04, -7.87666786e+04])

## linear regression intercept

In [64]:
reg.intercept_

1323627.7913782678